In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from OptimalObservableHelper import OptimalObservableHelper
from AltSetupHandler import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x9d0f630


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
# prod = False
prod = True
no_rvec = True
write_outputs = False
# write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/reweighted/test.json"
# output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/test"
# output_meta_path = "data/datasets/reweighted"
# output_meta = f"{output_meta_path}/test.json"
# checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = [
#     "true_lep_lvec", "true_nu_lvec", "true_quark1_lvec", "true_quark2_lvec",
#     "iso_lep_lvec", "nu_lvec", "R2Jet_sel1_lvec", "R2Jet_sel2_lvec",
#     ]
# true lvecs do not exist in every df so cannot be explicitly requested...
# urgh but empty snapshots are also not allowed
# output_collections = r"(true_\w+_lvec)|(iso_lep_lvec)|(nu_lvec)|(R2Jet_sel1_lvec)|(R2Jet_sel2_lvec)|(\w*sqme\w*)|(weight\w*)"
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/reweighted/signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    # output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/signal-only"
    # output_meta_path = "data/datasets/reweighted"
    # output_meta = f"{output_meta_path}/signal-only.json"
    # checked_output_meta = f"{output_meta_path}/checked-signal-only.json"
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
ROOT.EnableImplicitMT(n_threads)
# environ["OMP_NUM_THREADS"] = "6"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = OptimalObservableHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xddf8c60


In [7]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [8]:
oo_configs = [
    "g1z_pos_1em08",
    "ka_pos_1em08",
    "la_pos_1em08",
    ]
analysis.define_optimal_observables_truth(oo_configs, truth_categories=signal_category)
analysis.define_optimal_observables_reco(oo_configs)
oo_names = [f"O_{c}" for c in oo_configs]

In [9]:
analysis.book_histogram_1D("mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_ka_pos_1em08", "mc_O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_la_pos_1em08", "mc_O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)

analysis.book_histogram_1D("O_g1z_pos_1em08", "O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("O_ka_pos_1em08", "O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("O_la_pos_1em08", "O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)

In [10]:
alt_setup_handler = AltSetupHandler("""
{
  "SM": {
    "g1z": 1.0,
    "ka": 1.0,
    "la": 0.0
  },
"variations": [
    0.001,
    -0.001,
    1e-04,
    -1e-04,
    1e-08
  ]
}
""", mirror=False, combinations=False)
alt_config_names = list(alt_setup_handler.get_alt_setup().keys())

In [11]:
# book weighted OO sums
for name in ["nominal"] + alt_config_names:
    w_name = f"weight_{name}"
    analysis.book_sum(w_name, w_name, categories=signal_category)
    for oo in oo_names:
        oo_w_name = f"{oo}_{w_name}"
        analysis.define_only_on(signal_category, oo_w_name, f"{oo} * {w_name}")
        analysis.book_sum(oo_w_name, oo_w_name, categories=signal_category)
        analysis.book_histogram_1D(oo_w_name, oo_w_name, ("", "", 250, -2.5, 2.5), categories=signal_category)
        mc_oo_w_name = f"mc_{oo}_{w_name}"
        analysis.define_only_on(signal_category, mc_oo_w_name, f"mc_{oo} * {w_name}")
        analysis.book_sum(mc_oo_w_name, mc_oo_w_name, categories=signal_category)
        analysis.book_histogram_1D(mc_oo_w_name, mc_oo_w_name, ("", "", 250, -2.5, 2.5), categories=signal_category)
        print(mc_oo_w_name)

mc_O_g1z_pos_1em08_weight_nominal
mc_O_ka_pos_1em08_weight_nominal
mc_O_la_pos_1em08_weight_nominal


mc_O_g1z_pos_1em08_weight_g1z_pos_1em03
mc_O_ka_pos_1em08_weight_g1z_pos_1em03
mc_O_la_pos_1em08_weight_g1z_pos_1em03
mc_O_g1z_pos_1em08_weight_ka_pos_1em03
mc_O_ka_pos_1em08_weight_ka_pos_1em03
mc_O_la_pos_1em08_weight_ka_pos_1em03
mc_O_g1z_pos_1em08_weight_la_pos_1em03
mc_O_ka_pos_1em08_weight_la_pos_1em03
mc_O_la_pos_1em08_weight_la_pos_1em03
mc_O_g1z_pos_1em08_weight_g1z_neg_1em03
mc_O_ka_pos_1em08_weight_g1z_neg_1em03
mc_O_la_pos_1em08_weight_g1z_neg_1em03
mc_O_g1z_pos_1em08_weight_ka_neg_1em03
mc_O_ka_pos_1em08_weight_ka_neg_1em03
mc_O_la_pos_1em08_weight_ka_neg_1em03
mc_O_g1z_pos_1em08_weight_la_neg_1em03
mc_O_ka_pos_1em08_weight_la_neg_1em03
mc_O_la_pos_1em08_weight_la_neg_1em03
mc_O_g1z_pos_1em08_weight_g1z_pos_1em04
mc_O_ka_pos_1em08_weight_g1z_pos_1em04
mc_O_la_pos_1em08_weight_g1z_pos_1em04
mc_O_g1z_pos_1em08_weight_ka_pos_1em04
mc_O_ka_pos_1em08_weight_ka_pos_1em04
mc_O_la_pos_1em08_weight_ka_pos_1em04
mc_O_g1z_pos_1em08_weight_la_pos_1em04
mc_O_ka_pos_1em08_weight_la_pos_

In [12]:
if write_outputs:
    analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [13]:
%%time
analysis.run()

CPU times: user 9.9 s, sys: 288 ms, total: 10.2 s
Wall time: 3.68 s


In [14]:
# if write_outputs:
    # analysis.check_snapshots("events", output_path, checked_output_meta)

In [15]:
analysis.draw_histogram("mc_O_g1z_pos_1em08", categories=signal_category)
analysis.draw_histogram("O_g1z_pos_1em08", categories=signal_category)

analysis.draw_histogram("mc_O_ka_pos_1em08", categories=signal_category)
analysis.draw_histogram("O_ka_pos_1em08", categories=signal_category)

analysis.draw_histogram("mc_O_la_pos_1em08", categories=signal_category)
analysis.draw_histogram("O_la_pos_1em08", categories=signal_category)

(<cppyy.gbl.THStack object at 0x1341f2f0>,
 <cppyy.gbl.TCanvas object at 0x11dbecf0>)

In [16]:
analysis.compare_histograms_unscaled(["mc_O_g1z_pos_1em08", "O_g1z_pos_1em08"], categories=signal_category)
analysis.compare_histograms_unscaled(["mc_O_ka_pos_1em08", "O_ka_pos_1em08"], categories=signal_category)
analysis.compare_histograms_unscaled(["mc_O_la_pos_1em08", "O_la_pos_1em08"], categories=signal_category)

In [17]:
analysis.compare_histograms_unscaled(["mc_O_g1z_pos_1em08_weight_nominal"] + [f"mc_O_g1z_pos_1em08_weight_{c}" for c in alt_config_names if "g1z" in c], categories=signal_category)

In [18]:
# calculate means etc.
w_sums = {}
oo_sums = {}
mc_oo_sums = {}
oo_means = {}
mc_oo_means = {}
for name in ["nominal"] + alt_config_names:
    w_name = f"weight_{name}"
    w_sum = analysis.get_sum(w_name, categories=signal_category)
    w_sums[w_name] = w_sum
    for oo in oo_names:
        oo_w_name = f"{oo}_{w_name}"
        oo_sums[oo_w_name] = analysis.get_sum(oo_w_name, categories=signal_category)
        oo_means[oo_w_name] = oo_sums[oo_w_name] / w_sum
        mc_oo_w_name = f"mc_{oo}_{w_name}"
        mc_oo_sums[mc_oo_w_name] = analysis.get_sum(mc_oo_w_name, categories=signal_category)
        mc_oo_means[mc_oo_w_name] = mc_oo_sums[mc_oo_w_name] / w_sum

In [19]:
print(alt_config_names)

['g1z_pos_1em03', 'ka_pos_1em03', 'la_pos_1em03', 'g1z_neg_1em03', 'ka_neg_1em03', 'la_neg_1em03', 'g1z_pos_1em04', 'ka_pos_1em04', 'la_pos_1em04', 'g1z_neg_1em04', 'ka_neg_1em04', 'la_neg_1em04', 'g1z_pos_1em08', 'ka_pos_1em08', 'la_pos_1em08']


In [35]:
graphs = {}

for oo in oo_names:
    for par in ["g1z", "ka", "la"]:
        vars = [var for var in alt_config_names if par in var]
        nominal = mc_oo_means[f"mc_{oo}_weight_nominal"]
        x = [AltSetupHandler.get_var_from_name_1d(var) for var in vars]
        y = [mc_oo_means[f"mc_{oo}_weight_{var}"] / nominal for var in vars]
        x.append(0.)
        y.append(1.)
        vals = sorted(zip(x, y), key=lambda a: a[0])
        print(vals)
        g = ROOT.TGraph()
        for v in vals:
            g.AddPoint(v[0], v[1])
        g.SetTitle(f"{oo};{par}; #overline{{O}} / #overline{{O}}_{{0}}")
        graphs[f"mc_{oo}_{par}"] = g

[(-0.001, 1.0142352967922037), (-0.0001, 1.0014240967644696), (0.0, 1.0), (1e-08, 0.999999857584039), (0.0001, 0.9985757775344035), (0.001, 0.9857521331112545)]
[(-0.001, 0.9997536094735605), (-0.0001, 0.9999758828507093), (0.0, 1.0), (1e-08, 1.000000002405916), (0.0001, 1.0000240011601889), (0.001, 1.0002347916200651)]
[(-0.001, 0.9919053898223839), (-0.0001, 0.9991902370130306), (0.0, 1.0), (1e-08, 1.0000000809796432), (0.0001, 1.0008098298447616), (0.001, 1.0081012959499265)]
[(-0.001, 0.9996644796253297), (-0.0001, 0.9999641519897164), (0.0, 1.0), (1e-08, 1.0000000036103167), (0.0001, 1.000036358277026), (0.001, 1.0003865469957913)]
[(-0.001, 1.0063775313865346), (-0.0001, 1.0006368861763228), (0.0, 1.0), (1e-08, 0.9999999363210202), (0.0001, 0.9993633065414759), (0.001, 0.9936417403874166)]
[(-0.001, 0.9990128581929681), (-0.0001, 0.9998995538980134), (0.0, 1.0), (1e-08, 1.0000000100638529), (0.0001, 1.0001008309058572), (0.001, 1.0010256221433649)]
[(-0.001, 1.00932394155303), (-

In [36]:
canvases = {}
for k, g in graphs.items():
    c = ROOT.TCanvas()
    g.Draw("alp")
    c.Draw()
    canvases[k] = c